In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time

In [2]:
data=pd.read_csv('final_taylor_swift_lyrics.tsv',sep='\t')

In [3]:
START_P=data[data.line_number==0].index.tolist()

In [4]:

TERM=[]
for i in range(len(START_P)-1):
    LYR=np.array(data[START_P[i]:START_P[i+1]].lyric)
    term=LYR[0]+'\n '
    for t in LYR[1::]:
        term=term+t+ ' \n '
    TERM.append(term)

In [6]:
new_term=TERM[0]+' \n'
for t in TERM:
    new_term=new_term+t+' \n'

In [200]:
f = open("fuke.txt", "a")
f.write("Now the file has more content!")
f.close()

In [14]:
text = open('choruses.txt', 'rb').read().decode(encoding='utf-8')

In [66]:
text[0:5000]

"Can I go where you go?\n Can we always be this close? \n Forever and ever, ah \n Take me out, and take me home \n You're my, my, my, my lover \n Can I go where you go? \n Can we always be this close? \n Forever and ever, ah \n Take me out, and take me home \n You're my, my, my, my lover \n Ooh, look what you made me do \n Look what you made me do \n Look what you just made me do \n Look what you just made me— \n Ooh, look what you made me do \n Look what you made me do \n Look what you just made me do \n Look what you just made me do \n So oh-oh, oh-oh, oh-oh, oh-oh, oh-oh \n You need to calm down, you're being too loud \n And I'm just like oh-oh, oh-oh, oh-oh, oh-oh, oh-oh (Oh) \n You need to just stop, like can you just not step on my gown? \n You need to calm down \n In the middle of the night, in my dreams \n You should see the things we do, baby \n In the middle of the night, in my dreams \n I know I'm gonna be with you, so I take my time \n Are you ready for it? \n So it's gonna

In [16]:
S=[]
for l in data.lyric.tolist():
    if l[0]=='[':
        S.append(1)
    else:
        S.append(0)

In [17]:
data[data.lyric=='[Chorus]'].drop_duplicates(subset='song_title')

,index,album,song_title,lyric,line_number,release_date
5,0,Lover,Lover,[Chorus],5,2019-08-16
53,1,reputation,Look What You Made Me Do,[Chorus],14,2017-08-25
295,4,Lover,You Need To Calm Down,[Chorus],11,2019-06-14
346,5,reputation,...Ready for It?,[Chorus],14,2017-09-03
417,6,1989 (Deluxe),Blank Space,[Chorus],18,2014-10-27
...,...,...,...,...,...,...
15865,455,Live from Paris,Lover (Live from Paris),[Chorus],5,2020-05-17
16092,464,Teardrops On My Guitar - EP,Teardrops On My Guitar (Acoustic),[Chorus],16,2007-02-19
16133,466,Live from Paris,The Archer (Live from Paris),[Chorus],10,2020-05-17
16194,468,Live from Paris,You Need To Calm Down (Live from Paris),[Chorus],11,2020-05-17


In [18]:
data.lyric[0:4]

0                                            [Verse 1]
1    We could leave the Christmas lights up 'til Ja...
2             And this is our place, we make the rules
3    And there's a dazzling haze, a mysterious way ...
Name: lyric, dtype: object

In [19]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

73 unique characters


In [20]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [21]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)



In [23]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



In [24]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [25]:
# Batch size
BATCH_SIZE = 10

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((10, 100), (10, 100)), types: (tf.int64, tf.int64)>

In [26]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [28]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(10, 100, 73) # (batch_size, sequence_length, vocab_size)


In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (10, None, 256)           18688     
_________________________________________________________________
gru (GRU)                    (10, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (10, None, 73)            74825     
Total params: 4,031,817
Trainable params: 4,031,817
Non-trainable params: 0
_________________________________________________________________


In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [33]:
sampled_indices

array([27, 66, 68, 16, 14, 32, 32, 26,  6, 72, 52, 52,  5, 25, 18,  1, 30,
       43, 12, 31, 47, 65, 58, 40, 30, 70,  1, 15, 43, 60, 18, 65, 14, 10,
       10, 30, 50, 28, 26,  9, 33, 23, 44, 50, 36, 52, 33, 18,  6, 35, 14,
        8,  1,  8,  6, 35,  2, 19, 69, 49, 24,  0,  7, 57, 20, 71, 43, 42,
       29, 14, 37,  5, 24, 43, 60, 38, 68, 61, 19, 38, 30, 55,  7, 27,  3,
       16, 55, 22, 17, 45, 51, 42,  7, 53, 20, 29, 34, 21, 29, 18])

In [34]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)


In [35]:
model.compile(optimizer='adam', loss=loss)

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [37]:
EPOCHS = 50

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
40/40 [==============================] - 67s 2s/step - loss: 4.2276
Epoch 2/50
40/40 [==============================] - 65s 2s/step - loss: 2.5632
Epoch 3/50
40/40 [==============================] - 69s 2s/step - loss: 2.1601
Epoch 4/50
40/40 [==============================] - 78s 2s/step - loss: 2.0214
Epoch 5/50
40/40 [==============================] - 68s 2s/step - loss: 1.9208
Epoch 6/50
40/40 [==============================] - 82s 2s/step - loss: 1.8159
Epoch 7/50
40/40 [==============================] - 74s 2s/step - loss: 1.7215
Epoch 8/50
40/40 [==============================] - 66s 2s/step - loss: 1.6394
Epoch 9/50
40/40 [==============================] - 75s 2s/step - loss: 1.5663
Epoch 10/50
40/40 [==============================] - 70s 2s/step - loss: 1.4699
Epoch 11/50
40/40 [==============================] - 68s 2s/step - loss: 1.3822
Epoch 12/50
40/40 [==============================] - 67s 2s/step - loss: 1.2752
Epoch 13/50
40/40 [==============================

In [40]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_50'

In [41]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            18688     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 73)             74825     
Total params: 4,031,817
Trainable params: 4,031,817
Non-trainable params: 0
_________________________________________________________________


In [55]:
def generate_text(model, start_string,t):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 300

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [57]:
print(generate_text(model, start_string=u"Call me",t=0.3))

Call me 
 You’ll never leave me 
 And I’m not crazy and 
 This is really happening 
 That this is really happening 
 That this is really happening 
 That this is really happening 
 That this is really happening 
 That this is really happening 
 That this is really happening 
 That this is really happening 


In [59]:
print(generate_text(model, start_string=u"So", t=0.2))

So light 
 Or was it just like those promises that you made? On our last night 
 I said, that's when I could finally breathe 
 And by morning 
 Gone was any trace of your mind 
 I shouldn't be asking myself, "Why?" 
 You shouldn't be begging for forgiveness at my feet 
 You should've said no, you shou


In [62]:
print(generate_text(model, start_string=u"Why", t=0.8))

Why? 
 In the garden, would you trust me, me 
 Red lips and rosy cheek 
 Say a mind of my friends are saying 
 Girl, what are you thinking? 
 You're better off 
 You're better off, you're be the man 
 I'd be the man 
 Don'beat wet you clean 
 I still got you all over me 
 Drop everything now 
 Meet me 


In [65]:
print(generate_text(model, start_string=u"Let me", t=0.7))

Let me do 
 Look what you made me do 
 Look what you just made me— 
 Ooh, ooh, ooh, I love you I love you 
 And I'm down to my last grean 
 This is life before you know who you're gonna be 
 Fifteen 
 Drop everything now 
 Meet me in the pouring rain 
 Kiss me on the sidewalk 
 Take away the pain 
 'Cause
